In [1]:
# import YOLO model
from ultralytics import YOLO

In [2]:
# input size 640x640

In [3]:
# Load a model
model = YOLO('yolov8m-cls.pt') # load a pretrained model (recommended for training)

⚠️ Download failure, retrying 1/3 https://github.com/ultralytics/assets/releases/download/v0.0.0/yolov8m-cls.pt...
######################################################################## 100.0%


In [4]:
#model = YOLO(dotenv['MODEL_DIR'] / 'yolov8_moires_24-02-22.pt')

In [5]:
# Train the model
model.train(data='./dataset/', epochs=10)

New https://pypi.org/project/ultralytics/8.1.18 available 😃 Update with 'pip install -U ultralytics'
Ultralytics YOLOv8.0.184 🚀 Python-3.10.11 torch-2.0.1 CPU (Apple M1 Pro)
engine/trainer: task=classify, mode=train, model=yolov8m-cls.pt, data=./dataset/, epochs=10, patience=50, batch=16, imgsz=224, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=None, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, show=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, vid_stride=1, stream_buffer=False, line_width=None, visualize=False, augment=False, agnostic_nms=False, classes=

ultralytics.utils.metrics.ClassifyMetrics object with attributes:

confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x29d9d4340>
fitness: 0.9425086975097656
keys: ['metrics/accuracy_top1', 'metrics/accuracy_top5']
results_dict: {'metrics/accuracy_top1': 0.8850173950195312, 'metrics/accuracy_top5': 1.0, 'fitness': 0.9425086975097656}
save_dir: PosixPath('/Users/frederic.birwe/Desktop/test/pagina-finder-webapp/runs/classify/train22')
speed: {'preprocess': 0.0008564792859014318, 'inference': 285.43388137418634, 'loss': 0.00014496181900077582, 'postprocess': 0.0001287626472499728}
top1: 0.8850173950195312
top5: 1.0

In [6]:
metrics = model.val() # no arguments needed, dataset and settings remembered
metrics.top1 # top1 accuracy
metrics.top5 # top5 accuracy

Ultralytics YOLOv8.0.184 🚀 Python-3.10.11 torch-2.0.1 CPU (Apple M1 Pro)
YOLOv8m-cls summary (fused): 103 layers, 15765218 parameters, 0 gradients, 41.6 GFLOPs
train: /Users/frederic.birwe/Documents/GitHub/master-thesis_moire_detection/60_gebastel/Model/dataset/train... found 5051 images in 2 classes ✅ 
val: /Users/frederic.birwe/Documents/GitHub/master-thesis_moire_detection/60_gebastel/Model/dataset/val... found 574 images in 2 classes ✅ 
test: /Users/frederic.birwe/Documents/GitHub/master-thesis_moire_detection/60_gebastel/Model/dataset/test... found 1276 images in 2 classes ✅ 
val: Scanning /Users/frederic.birwe/Documents/GitHub/master-thesis_moire_detec
               classes   top1_acc   top5_acc: 100%|██████████| 36/36 [06:04<00
                   all      0.885          1
Speed: 0.0ms preprocess, 627.1ms inference, 0.0ms loss, 0.0ms postprocess per image
Results saved to /Users/frederic.birwe/Desktop/test/pagina-finder-webapp/runs/classify/val25


1.0

In [7]:
str(metrics.save_dir)

'/Users/frederic.birwe/Desktop/test/pagina-finder-webapp/runs/classify/val25'

In [8]:
!open '/Users/frederic.birwe/Desktop/test/pagina-finder-webapp/runs/classify/val19'

In [9]:
metrics.confusion_matrix.matrix

array([[        222,          15],
       [         51,         286]])

In [10]:
from pathlib import Path
import torch
import plotly.express as px
from tqdm.auto import tqdm

In [11]:
from PIL import Image

In [12]:
import sys
sys.path.append('../../30_data_tools/')
from helper import load_dotenv

In [13]:
dotenv = load_dotenv()

In [14]:
model = YOLO(dotenv['MODEL_DIR'] / 'yolov8_moires_24-02-22.pt')

In [35]:
moire_img = list( Path('./dataset/test/moire/').glob('./*.jpg') )
non_moire_img = list( Path('./dataset/test/no_moire/').glob('./*.jpg') )

data = moire_img + non_moire_img
labels = [1 for i in moire_img] + [0 for i in non_moire_img]

In [36]:
#model.val()
predictions = []
target_labels = []

for entry in tqdm(data):
    pred = model.predict(entry, verbose=False)[0].probs.data
    predictions.append(pred)
    target_labels.append( 1 - int(torch.argmax(pred)) )

  0%|          | 0/1276 [00:00<?, ?it/s]

In [17]:
str(metrics.save_dir)

'/Users/frederic.birwe/Desktop/test/pagina-finder-webapp/runs/classify/val25'

In [37]:
TP = 0
TN = 0
FP = 0
FN = 0

for i in range(len(labels)):
    # Positive == 1, Negative == 0
    res = labels[i] - target_labels[i]
    if res == 0:
        if labels[i] == 1:
            TP += 1
        else:
            TN += 1
    elif res == -1:
        FP += 1
    else:
        FN += 1

recall = TP / (TP + FN)
precision = TP / (TP + FP)
correct_values = TP + TN
accuracy = correct_values / len(labels)

In [38]:
TP, TN, FP, FN

(483, 649, 17, 127)

In [39]:
TP / (TP + FN)

0.7918032786885246

In [40]:
recall, precision

(0.7918032786885246, 0.966)

In [41]:
f"{(TP + TN)}/{len(labels)}, {accuracy}"

'1132/1276, 0.8871473354231975'

In [23]:
import shutil

In [24]:
for i in tqdm(range(1,len(labels))):
    # Positive == 1, Negative == 0
    res = labels[i] - target_labels[i]
    if res == 1:
        dir_path = Path('./temp/FN/')

        if dir_path.exists() == False:
            dir_path.mkdir()
    elif res == -1:
        dir_path = Path('./temp/FP/')

        if dir_path.exists() == False:
            dir_path.mkdir()

    if res != 0:
        out_path = dir_path / data[i].name
        shutil.copy( data[i], out_path )

  0%|          | 0/3915 [00:00<?, ?it/s]

In [25]:
from random import randrange

In [26]:
model.predict(data[i], verbose=False)[0].probs.data

tensor([0.2409, 0.7591])

In [27]:
model.predict(Path('./dataset/train/moire/508890.HBZP_1_M_270.p1.soft_light.1.4c_600.jpg.0908.jpg'), verbose=False)[0].probs.data

FileNotFoundError: dataset/train/moire/508890.HBZP_1_M_270.p1.soft_light.1.4c_600.jpg.0908.jpg does not exist

In [ ]:
classes = {
    '1' : [],
    '0.5' : [],
    '-0.5' : [],
    '-1' : []
}

for i in range(len(predictions)):
    diff = float(predictions[i][0] - predictions[i][1])

    if diff > 0.5:
        classes['1'].append(i)
    elif diff >= 0:
        classes['0.5'].append(i)
    elif diff >= -.5:
        classes['-0.5'].append(i)
    else:
        classes['-1'].append(i)

In [ ]:
for c in classes:
    acc = len([True for i in classes[c] if labels[i] - target_labels[i] == 0]) / len(classes[c])

    print(c, acc)

In [ ]:
i = randrange(len(data))
print(i, data[i].name)
print(predictions[i], labels[i],target_labels[i])
Image.open(data[i])

In [ ]:
import pandas as pd

In [ ]:
res_df = pd.DataFrame(
    [(float(predictions[i][0]),float(predictions[i][1]),labels[i],target_labels[i]) for i in range(len(labels))],
    columns=['pred_moire','pred_no_moire','label','target_label']
)

res_df.loc[
    :,
    'pred_diff'
] = res_df.pred_moire - res_df.pred_no_moire
res_df.loc[
    :,
    'pred_correct'
] = res_df.label == res_df.target_label
res_df.loc[
    :,
    'pred_diff_abs'
] = res_df.pred_diff.abs()

In [ ]:
res_df.corr()

In [ ]:
px.scatter(
    y=res_df.pred_diff_abs,
    color=res_df.pred_correct
)

In [ ]:
px.scatter(
    sorted([p[0]-p[1] for p in predictions], reverse=True)
)

In [ ]:
px.scatter(
    [p[0] for p in predictions]
)

In [ ]:
px.scatter(
    [p[1] for p in predictions]
)